二叉堆的本质就是二叉树，其最主要作用在于能够随着插入数据动态调整树中元素的排列顺序，为了优化其时间复杂度，我们将二叉堆用一个数组来实现，这只需要保证二叉树是完全二叉树，就可以按照树中从上到下从左到右的顺序将元素映射到数组中，而且可以方便地根据一个节点的索引值找到其父节点和左右子节点的索引值。

二叉堆动态调整的规则如下：（以小顶堆为例）

1. 增：push/swim插入元素

    1、先把新元素追加到二叉树底层的最右侧，保持**完全二叉树**的结构。此时该元素的父节点可能比它大，不满足小顶堆的性质。

    2、为了恢复小顶堆的性质，需要将这个新元素不断上浮（swim），直到它的父节点比它小为止，或者到达根节点。此时整个二叉树就满足小顶堆的性质了。

2. 删：pop/sink删除元素
   
    1、先把堆顶元素删除，把二叉树底层的最右侧元素摘除并移动到堆顶，保持**完全二叉树**的结构。此时堆顶元素可能比它的子节点大，不满足小顶堆的性质。

    2、为了恢复小顶堆的性质，需要将这个新的堆顶元素不断下沉（sink），直到它比它的子节点小为止，或者到达叶子节点。此时整个二叉树就满足小顶堆的性质了。

下面是一个简化版的小顶堆优先级队列的核心逻辑实现（没有处理特殊情况以及优化处理）

In [4]:
class SimpleMinPQ:
    # 二叉堆使用数组实现
    def __init__(self, capacity):
        self.heap = [0] * capacity
        self.size = 0
    
    # 父节点的索引
    def parent(self, node):
        return (node - 1) // 2
    
    # 左子节点的索引
    def left(self, node):
        return node * 2 + 1
    
    # 右子节点的索引
    def right(self, node):
        return node * 2 + 2
    
    # 查，返回堆顶元素，O(1)
    def peek(self):
        return self.heap[0]
    
    def swap(self, i, j):
        self.heap[i], self.heap[j] = self.heap[j], self.heap[i]
    
    # 增，向堆中插入一个元素，O(logN)
    def push(self, x):
        # 将新元素插入在二叉树底层的最右侧，即数组heap的末端
        self.heap[self.size] = x
        # 上浮到正确位置
        self.swim(self.size)
        self.size += 1
    
    # 删，删除堆顶元素，O(logN)
    def pop(self):
        res = self.heap[0]
        # 把堆底元素放到堆顶
        self.heap[0] = self.heap[self.size - 1] 
        self.size -= 1
        # 下沉到正确位置
        self.sink(0)
        return res
    
    # 定义上浮，时间复杂度等于树高O(logN)
    def swim(self, node):
        while node > 0 and self.heap[self.parent(node)] > self.heap[node]:
            self.swap(self.parent(node), node)
            node = self.parent(node)
    
    # 定义下沉，时间复杂度等于树高O(logN)
    def sink(self, node):
        while self.left(node) < self.size:
            min_node = min(self.heap[node], self.heap[self.left(node)], self.heap[self.right(node)])
            if min_node == self.heap[node]:
                break
            elif min_node == self.heap[self.left(node)]:
                self.swap(self.left(node), node)
                node = self.left(node)
            elif min_node == self.heap[self.right(node)]:
                self.swap(self.right(node), node)
                node = self.right(node)
if __name__ == "__main__":
    pq = SimpleMinPQ(5)
    pq.push(3)
    pq.push(2)
    pq.push(1)
    pq.push(5)
    pq.push(4)

    print(pq.pop())  # 1
    print(pq.pop())  # 2
    print(pq.pop())  # 3
    print(pq.pop())  # 4
    print(pq.pop())  # 5        
                

1
2
3
4
5


基于简化版优先级队列，可以加入泛型、自定义比较器、扩容等功能，从而实现完善版优先级队列

In [5]:
class MyPriorityQueue:
    # 二叉堆使用数组实现
    def __init__(self, capacity, comparator=None):
        self.heap = [0] * capacity
        self.size = 0
        # 元素比较器
        self.comparator = comparator if comparator is not None else lambda x, y: (x > y) - (x < y)
    
    def size(self):
        return self.size
    def is_empty(self):
        return self.size == 0
    
    # 父节点的索引
    def parent(self, node):
        return (node - 1) // 2
    
    # 左子节点的索引
    def left(self, node):
        return node * 2 + 1
    
    # 右子节点的索引
    def right(self, node):
        return node * 2 + 2
    
    # 查，返回堆顶元素，O(1)
    def peek(self):
        if self.is_empty():
            raise IndexError("Priority queue underflow")
        return self.heap[0]
    
    def swap(self, i, j):
        self.heap[i], self.heap[j] = self.heap[j], self.heap[i]
    
    # 增，向堆中插入一个元素，O(logN)
    def push(self, x):
        # 扩容
        if self.size == len(self.heap):
            self.resize(2 * len(self.heap))

        # 将新元素插入在二叉树底层的最右侧，即数组heap的末端
        self.heap[self.size] = x
        # 上浮到正确位置
        self.swim(self.size)
        self.size += 1
    
    # 删，删除堆顶元素，O(logN)
    def pop(self):
        if self.is_empty():
            raise IndexError("Priority queue underflow")
        res = self.heap[0]
        # 把堆底元素放到堆顶
        self.heap[0] = self.heap[self.size - 1] 
        self.size -= 1
        # 下沉到正确位置
        self.sink(0)
        return res
    
    # 定义上浮，时间复杂度等于树高O(logN)
    def swim(self, node):
        while node > 0 and self.heap[self.parent(node)] > self.heap[node]:
            self.swap(self.parent(node), node)
            node = self.parent(node)
    
    # 定义下沉，时间复杂度等于树高O(logN)
    def sink(self, node):
        while self.left(node) < self.size:
            min_node = min(self.heap[node], self.heap[self.left(node)], self.heap[self.right(node)])
            if min_node == self.heap[node]:
                break
            elif min_node == self.heap[self.left(node)]:
                self.swap(self.left(node), node)
                node = self.left(node)
            elif min_node == self.heap[self.right(node)]:
                self.swap(self.right(node), node)
                node = self.right(node)

    def resize(self, capacity):
        new_heap = [None] * capacity
        for i in range(self.size):
            new_heap[i] = self.heap[i]
        self.heap = new_heap

# 测试代码
if __name__ == "__main__":
    # 小顶堆
    pq = MyPriorityQueue(3, comparator=lambda x, y: (x > y) - (x < y))
    pq.push(3)
    pq.push(1)
    pq.push(4)
    pq.push(1)
    pq.push(5)
    pq.push(9)

    # 1 1 3 4 5 9
    while not pq.is_empty():
        print(pq.pop())

1
1
3
4
5
9
